# Faraday's Law

M&I 

Ch 22 Faraday's Law

- §22.2 &ndash; Faraday's Law


In [1]:
########
## Env Setup / Preamble
########
%reload_ext autoreload
%autoreload complete 

%reload_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]
>
>   In the figure below, the solenoid radius is 4 [cm] and the ring radius is 
>   17 [cm]. $B = 0.5 \left[\textrm{T}\right]$ inside the solenoid, and $B \approx 0$ 
>   outside the solenoid.
>
>   
>
>   __What is the magnetic flux through the outer ring?__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{T}\cdot\textrm{m}^2\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

In [3]:
R_inner = 4e-2
R_i = R_inner

R_outer = 17e-2
R_o = R_outer

B = 0.5
flux = B * (π * R_i^2) # d_t B

print(flux)
print(n(flux))


0.000800000000000000*π
0.00251327412287183


<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]
>   The north pole of a bar magnet points toward a thing circular coil of wire 
>   containing 30 turns (see the figure). 
>   The magnet is moved away from the coil, so that the flux through one turn, 
>   inside the coil, decreases by $0.1 \left[\textrm{Tesla} \cdot \textrm{m}^2\right]$ 
>   in $0.2$ seconds.
>
>   
>
>   __What is the average $\mathcal{EMF}$ in the coil during this time interval?__
>
>   &emsp;&emsp;&emsp;&emsp; $\mathcal{EMF} = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>   
>   __Viewed from the right side (opposite the bar magnet), does the current run clockwise or counter-clockwise?__
>
>   - [ ] &nbsp; Clockwise
>
>   - [ ] &nbsp; Counter-Clockwise
>
>
>   __<p align='right'>(0.1 points)</p>__
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

In [4]:
N = 30
dB = -0.1
dt = 0.2

emf = N * (-dB / dt)

print(n(emf))


15.0000000000000


- [x] &emsp; clockwise

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]*</h2>

>   ---
>   ## [3]*
>
>   The magnetic field in a solenoid is $B = \left( \mu_0 N I \right) / d$.
>
>   A circular wire of radius 13 [cm] is concentric with a solenoid of radius 
>   $2$ [cm] and length $d = 1$ meter, containing $N = 11,000$ turns.  
>   The current increases at a rate of 41 [A/s]. 
>
>   __<p align='right'>(10 points)</p>__
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='3a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the $\mathcal{EMF}$ in the wire?__ 
>
>   &emsp;&emsp;&emsp;&emsp; $\mathcal{EMF} = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [5]:
R_o = 13e-2
R_i = 2e-2

d = 1
N = 11_000

I = 41

B_dt = (μ_0 * N * I) / d

emf = B_dt * (π * R_i^2)

n(emf)


0.000712190653488576

<h3 id='3b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the non-Coulomb electric field in the wire?__
>
>   &emsp;&emsp;&emsp;&emsp;$\fbox{\hspace{8em}?}\ \left[ \textrm{C} \right]$
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [6]:
efield = emf / (2*π*R_o)

n(efield)


0.000871912791742723

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]*</h2>

>   ---
>   ## [4]* - A current-carrying wire moves toward a coil
>
>   A long straight wire carrying current $I$, is moving with speed $v$ toward a
>   small circular coil of radius $r$ containing $N$ turns; which is attached to
>   a voltmeter as shown.  The long wire is in the plane of the coil (only a 
>   small portion of the wire is shown in the diagram).
>
>   
>
>   The radius of the coil is 0.02 m, and the coil has 11 turns.  At a particular 
>   instant, $I = 5$ amperes, $v = 3.8$ meters per second, and the distance from 
>   the wire to the center of the coil is $x = 0.16$ meters. 
>   __<p align='right'>(10 points)</p>__
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the _magnitude_ of the rate of change of the magnetic field inside the coil?__
>   
>   &emsp;&emsp;&emsp;&emsp;$\frac{d\mathbf{B}}{dt} = \fbox{\hspace{8em}?}\ \left[\textrm{T} / \textrm{s}\right]$
>
>   You will need to derive a formula using calculus before you can get a number. 
>   Begin with the expression for the magnetic field due to the wire at the location of the coil. 
>   Use the approximate formula, since the wire is very long. 
>   Remember the chain rule, and remember that $v = \frac{dx}{dt}$. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

from defn $\mathcal{EMF} = -\int \frac{\partial \mathrm{\vec{B}}}{\partial t} \cdot \partial \mathrm{\vec{A}}$
$$
    \mathcal{EMF} = \frac{\Delta \Phi_B}{\Delta t} \leftrightarrow \frac{d}{dt} B \cdot A \\
    \Rightarrow \mathcal{EMF} = \frac{d \Phi_B}{d t} \leftrightarrow \frac{d}{dt} \frac{\mu_0 I}{2\pi x} \cdot A  \\
$$

now, if

$B \approx \frac{\mu_0 I}{2\pi R} = \frac{\mu_0 I}{2\pi x_t}$, then;
$\frac{dB}{dt} \approx -\frac{\mu_0 I}{2\pi x^{2}} \frac{dx}{dt} = -\frac{\mu_0 Iv}{2\pi x^{2}} $

In [ ]:
N = 11
I = 5
v = 2.6
x = 12e-2

dB_dt = -μ_0 * I * v / (2 * π * x^2)

print(dB_dt)
n(dB_dt)


-451.388888888889*μ_0/π


-0.000180555555531716

<h3 id='4b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the magnitude of the Voltmeter reading?__ 
>   Remember that this includes all 11 turns of the coil.
>
>   &emsp;&emsp;&emsp;&emsp;$\fbox{\hspace{8em}?}\ \left[\textrm{Volts}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [ ]:
r_coil = 2e-2

emf = -N * dB_dt * π * r_coil^2

print(n(emf))


2.49582083002236e-6


<h3 id='4c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the direction of vector $\frac{d \mathbf{\vec{B}}}{dt}$?__
>
>   __Select one:__
>
>   - [ ] &nbsp; left
>
>   - [ ] &nbsp; right
>
>   - [ ] &nbsp; zero -- no direction
>
>   - [ ] &nbsp; up
>
>   - [ ] &nbsp; down
>
>   - [ ] &nbsp; into the page
>
>   - [ ] &nbsp; out of the page
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] into the page

<h3 id='4d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the direction of the curly electric field in the coil?__
>
>   __Select one:__
>
>   - [ ] &nbsp; $\mathbf{E}=0$ in the coil.
>
>   - [ ] &nbsp; clockwise
>
>   - [ ] &nbsp; counter-clockwise
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] counter-clockwise

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]</h2>

>   ---
>   ## [5]
>
>   In the figure below, a thin coil of radius $r_1 = 4$ cm, containing 
>   $N_1 = 5000$ turns, is connected through a resistor $R = 100\ \Omega$, to an
>   AC power supply running at a frequency $f = 2500$; so that the current through 
>   the resistor (and coil) is $I(t) = I_1 \sin\left(2\pi \cdot 2500\ t\right)$. 
>
>   The voltage across the resistor triggers an oscilloscope that also displays 
>   the voltage, which is 10 V peak-to-peak; and therefore has an amplitude of 
>   $5$ [V] (which indicates the amplitude of the current is $I_1 = 0.05$ [A]). 
>
>   
>
>   A second thin coil of radius $r_2 = 2$ cm, containing $N_2 = 3200$ turns, is
>   a distance $L = 40$ cm from the first coil. The axes of the two coils are 
>   along the same line.  The second coil is connected to the upper input of the
>   oscilloscope, so that the voltage across the second coil can be displayed 
>   along with the voltage across the resistor. 
>
>   <hr width="82%" style="border-top:1px dotted black; border-bottom:none"/>
>
>   __Calculate the amplitude (maximum voltage) of the second coil voltage.__
>
>   &emsp;&emsp;&emsp;&emsp;$\fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>   
>   __If you must make simplifying assumptions, state clearly what they are:__ 
>
>   (mark one of the following options:)
>   
>   - [ ] &nbsp; Our assumptions are that $L \ll r_1$, and that the coils are **thick** coils. 
>   - [ ] &nbsp; Our assumptions are that $L \gg r_1$, and that the coils are **thin** coils. 
>
>   __<p align='right'>(0.1 points)</p>__
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

$$
    \mathcal{EMF} = - N \int \frac{\partial \mathbf{\vec{B}}}{\partial t} \cdot \partial \mathbf{\vec{A}} \\
    \Rightarrow \mathcal \approx - \frac{dB}{dt} \cdot A \\
$$

then, we treat solenoid coil as a magnetic dipole with a related moment $\vec{\mu} = NI \vec{A}$, so we can we can use the on-axis approximation for the magnetic field of a dipole (c.ref §17.10)

$$
    B_\textrm{axis} \approx \frac{\mu_0}{4\pi} \frac{2\vec{\mu}}{\|r\|^3} \\
    \Rightarrow \frac{\mu_0}{4\pi} 2 N \frac{I \vec{A}}{\|r\|^3}
$$

In [14]:
r_1 = 4e-2
N_1 = 5_000

f_1 = 2500
I_1 = 0.05

r_2 = 2e-2
N_2 = 3_200

L = 40e-2

B_2 = (μ_0 / (2 * π)) * N_1 * I_1 * (π * r_1^2) / L^3

# TODO :> there's hidden calculus here to show

B2_dt = (μ_0) * N_1 * ((π * r_1^2) / L^3) * (I_1 * f_1)

emf_2 = -N_2 * B2_dt * (π * r_2^2)
print(n(abs(emf_2)))


0.248050213409648


- [x] &nbsp; $L \gg r_1$

<div style="page-break-after: always;"/> 

<h2 id='6' style="display:none">[6]*</h2>

>   ---
>   ## [6]*
>
>   Tall towers support power lines $h = 55$ [m] above the ground and $l = 19$ [m]
>   apart, that run from a hydroelectric plant to a large city.  The power lines
>   carry 60 [Hz] alternating current with amplitude $5 \times 10^4$ [A] (see 
>   figure below); i.e., the current in both of the power lines is 
>   $I = \left(5 \times 10^4\ [\textrm{A}]\right) \sin \left( 2\pi \cdot 60 \left[\textrm{Hz}\right] \cdot t \right)$. 
>
>   
>
>   __<p align='right'>(10 points)</p>__
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Calculate the amplitude (largest magnitude) and direction of the magnetic field__, 
>   produced by the two power lines at the base of the tower; when a current of 
>   $5 \times 10^4\ [\textrm{A}]$ in the "left" power line is headed out of the 
>   page, and a current of $5 \times 10^4\ [\textrm{A}]$ in the right power line
>   is headed into the page. 
>
>   &emsp;&emsp;&emsp;&emsp; __magnitude__ = $\fbox{\hspace{8em}?}\ \left[\textrm{T}\right]$
>
>   &emsp;&emsp;&emsp;&emsp; __direction__ (select one):
>
>   - [ ] &nbsp; up
>
>   - [ ] &nbsp; down
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [23]:
h = 55
l = 19
f = 60

I_1 = 5e4
I_2 = -5e4

x = vector(r_[0,0,0])
x1 = vector(r_[-l/2, h, 0])
x2 = vector(r_[l/2, h, 0])

r1 = ((x - x1).cross_product(vector(r_[0,0,1])))
r2 = ((x - x2).cross_product(vector(r_[0,0,1])))

B_1 = ((μ_0 * I_1) / (2*pi*r1.norm())) * r1 / r1.norm()
B_2 = ((μ_0 * I_2) / (2*pi*r2.norm())) * r2 / r2.norm()

B = (B_1 + B_2).norm()

n(B)


0.0000609902896958234

- [x] up

<h3 id='6b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(B)
>
>   This magnetic field is not large compared to the Earth's magnetic field, but
>   it varies in time, and so might have different biological effects than the 
>   Earth's steady field. 
>
>   __For a person laying on the ground at the base of the tower, approximately what is the maximum $\mathcal{EMF}$&nbsp;__ 
>   produced around the perimeter of the body (which is about 2 m long, by half a meter wide?)
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{mV}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [29]:
# TODO :> use vector Area w/ cross products to get y-hat for correct vec math

length = 2
width = 1/2
A = length * width
N = 1

dB_dt_1 = ((μ_0 * I_1 * f) / (r1.norm())) * (r1 / r1.norm())
dB_dt_2 = ((μ_0 * I_2 * f) / (r2.norm())) * (r2 / r2.norm())

dB_dt = (dB_dt_1 + dB_dt_2)

emf = -N * dB_dt * A
n(emf.norm() * 1e3) # convert from [V] to [mV]


22.9927975258454

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]*</h2>

>   ---
>   ## [7]*
>
>   A bar magnet whose dipole moment is $\langle 0, 0, 6 \rangle\ \left[\textrm{A} \cdot \textrm{m}^2\right]$ 
>   has a constant velocity of $\langle 0, 0, 5 \rangle$ [m/s]. 
>
>   When the center of the magnet is at location $\langle 1, 3, 4 \rangle$ [m], 
>   __what is the (vector) electric field at location $\langle\ 1.11, 3, 3\ \rangle$ [m]?__
>
>   &emsp;&emsp;&emsp;&emsp; $\mathbf{\vec{E}} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ \Big\rangle\ \left[\textrm{N}/\textrm{C}\right]$
>
>   __<p align='right'>(10 points)</p>__
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

In [ ]:
μ = vector(r_[0, 0, 6])
x_magnet = vector(r_[1, 3, 4])
v_magnet = vector(r_[0, 0, 5])

x = vector(r_[1.11, 3, 3])



---